In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Will it Saturate?

> Just a test to see if we are abe to saturate 1Gbit with file responsens using just pure Python without nginx or something like that. 

This file will become your README and also the index of your documentation.

## Install

`pip install will_it_saturate`

## How to use

Start control server

`wis_control_server`

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

### Create Benchmark Server

create server control client + server (needed for turning files into urls)

In [ ]:
# dont_test

server_control_host = Host(name="localhost", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(
    servers.FastAPIUvicornServer(host="localhost", port=5001)
)

<IPython.core.display.Javascript object>

### Create Files and Urls

In [ ]:
# dont_test

byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)
files = server_control_client.get_or_create_files(epoch)
epoch.urls = [server.file_to_url(file) for file in files]

<IPython.core.display.Javascript object>

### Create Control Client

In [ ]:
# dont_test

client_control_host = Host(name="localhost", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_client = clients.HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

### Create Result

In [ ]:
# dont_test

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

### Measure Server with Client

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)

<IPython.core.display.Javascript object>

### Save Result

We are using a sqlite database for saving benchmark results.

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results.db")
register_default_tables(repository)
results = repository.tables["result"]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result_id = results.add_result(result)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results_from_database = results.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)

,server,client,elapsed,file_size_h,bytes_per_second_h
0,fastAPI/uvicorn,httpx,4.318646,9.54MB,82.81MB
1,fastAPI/uvicorn,httpx,3.168980,9.54MB,112.85MB


<IPython.core.display.Javascript object>